## Scikit-Learn Clustering Support Vector Machines Model Testing

The following example will:

* Upload and convert a 

In [1]:
import json
import os
import pickle

import wallaroo
from wallaroo.pipeline   import Pipeline
from wallaroo.deployment_config import DeploymentConfigBuilder
from wallaroo.framework import Framework

import pyarrow as pa
import numpy as np
import pandas as pd

from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression

wl = wallaroo.Client(auth_type="sso", interactive=True)

In [2]:
def get_workspace(name):
    workspace = None
    for ws in wl.list_workspaces():
        if ws.name() == name:
            workspace= ws
    if(workspace == None):
        workspace = wl.create_workspace(name)
    return workspace

In [3]:
workspace = get_workspace("sklearn-cluster-svm-jch")
wl.set_current_workspace(workspace)

{'name': 'sklearn-cluster-svm-jch', 'id': 39, 'archived': False, 'created_by': '3cc9e92a-fa3c-4371-a7a7-487884df059e', 'created_at': '2023-06-15T20:44:01.964595+00:00', 'models': [{'name': 'logreg-test', 'versions': 1, 'owner_id': '""', 'last_update_time': datetime.datetime(2023, 6, 15, 20, 44, 2, 135851, tzinfo=tzutc()), 'created_at': datetime.datetime(2023, 6, 15, 20, 44, 2, 135851, tzinfo=tzutc())}], 'pipelines': [{'name': 'sklearn-cluster-svm-pipeline', 'create_time': datetime.datetime(2023, 6, 15, 20, 45, 2, 373884, tzinfo=tzutc()), 'definition': '[]'}]}

## Data & Model Creation

In [4]:
input_schema = pa.schema([
    pa.field('sepal length (cm)', pa.float64()),
    pa.field('sepal width (cm)', pa.float64()),
    pa.field('petal length (cm)', pa.float64()),
    pa.field('petal width (cm)', pa.float64())
])

output_schema = pa.schema([
    pa.field('output', pa.float64())
])

## Upload model

In [5]:
model = wl.upload_model('sklearn-svm', 'models/model-auto-conversion_sklearn_svm_pipeline.pkl', framework=Framework.SKLEARN, input_schema=input_schema, output_schema=output_schema)

model

Waiting for model conversion... It may take up to 10.0min.
Model is Pending conversion..Converting.Pending conversion..Converting.......Ready.


{'name': 'sklearn-svm', 'version': 'bdbf0c44-5a26-4cb3-a815-a4e8201299b9', 'file_name': 'model-auto-conversion_sklearn_svm_pipeline.pkl', 'image_path': 'proxy.replicated.com/proxy/wallaroo/ghcr.io/wallaroolabs/mlflow-deploy:v2023.3.0-main-3367', 'last_update_time': datetime.datetime(2023, 6, 15, 21, 42, 47, 600757, tzinfo=tzutc())}

## Configure model and pipeline

In [6]:
deployment_config = DeploymentConfigBuilder() \
    .cpus(0.25).memory('1Gi') \
    .build()

In [7]:
pipeline_name = f"sklearn-cluster-svm-pipeline"
pipeline = wl.build_pipeline(pipeline_name)
pipeline.add_model_step(model)

pipeline.deploy(deployment_config=deployment_config)
pipeline.status()

Waiting for deployment - this will take up to 90s ........... ok


{'status': 'Running',
 'details': [],
 'engines': [{'ip': '10.244.0.220',
   'name': 'engine-848c6d6bb7-6xvfb',
   'status': 'Running',
   'reason': None,
   'details': [],
   'pipeline_statuses': {'pipelines': [{'id': 'sklearn-cluster-svm-pipeline',
      'status': 'Running'}]},
   'model_statuses': {'models': [{'name': 'sklearn-svm',
      'version': 'bdbf0c44-5a26-4cb3-a815-a4e8201299b9',
      'sha': 'c6eec69d96f7eeb3db034600dea6b12da1d2b832c39252ec4942d02f68f52f40',
      'status': 'Running'}]}}],
 'engine_lbs': [{'ip': '10.244.1.44',
   'name': 'engine-lb-584f54c899-r4wp2',
   'status': 'Running',
   'reason': None,
   'details': []}],
 'sidekicks': [{'ip': '10.244.0.219',
   'name': 'engine-sidekick-sklearn-svm-32-7885f56cf9-249np',
   'status': 'Running',
   'reason': None,
   'details': [],
   'statuses': '\n'}]}

## Inference

In [8]:
pipeline.infer_from_file('data/test_cluster-svm.json')

,time,in.petal length (cm),in.petal width (cm),in.sepal length (cm),in.sepal width (cm),out.output,check_failures
0,2023-06-15 21:43:03.450,1.4,0.2,5.1,3.5,0.0,0
1,2023-06-15 21:43:03.450,1.4,0.2,4.9,3.0,0.0,0


In [9]:
pipeline.undeploy()

Waiting for undeployment - this will take up to 45s ....................................... ok


name,sklearn-cluster-svm-pipeline
created,2023-06-15 20:45:02.373884+00:00
last_updated,2023-06-15 21:42:52.006815+00:00
deployed,False
tags,
versions,"ad2fe461-88c7-4bb8-985c-9170783d667b, bf498d28-5231-4fc8-82ff-8c6f3e343848, 334ea3d3-9a51-402e-b8f4-0baaa522e3ea, 6b12b501-dcf0-4ac6-b8ab-a1ba90f2b966"
steps,logreg-test
